## Extract Earth System Science Data (ESSD) datasets from S3 into Pachyderm

Copyright (C) 2021 OS-Climate

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Contributed by Michael Tiemann (Github: MichaelTiemannOSC)

In [ ]:
import os
import boto3
import openpyxl

import osc_ingest_trino as osc

import python_pachyderm
import grpc

import io

# from pandas_profiling import ProfileReport  # Not compatible with Pydantic >=2.0

Load Credentials

In [ ]:
# From the AWS Account page, copy the export scripts from the appropriate role using the "Command Line or Programmatic Access" link
# Paste the copied text into ~/credentials.env

# Load environment variables from credentials.env
osc.load_credentials_dotenv()

Create an S3 resource for the bucket holding source data

In [ ]:
s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
# Create a pachyderm client by manually setting up host and port
try:
    client = python_pachyderm.Client(os.environ['PACH_ENDPOINT'], os.environ['PACH_PORT'])
    client.health_check()
except grpc._channel._InactiveRpcError:
    client = python_pachyderm.Client("localhost", os.environ['PACH_PORT'])

In [ ]:
client.delete_repo("essd")

In [ ]:
# Create a new pachyderm repo called `essd`
client.create_repo("essd")

In [ ]:
# Display list of repos
print(list(client.list_repo()))

In [ ]:
s3_client = boto3.client('s3',
                         aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
                         aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],)

with client.commit("essd", "master") as commit:
    for o in bucket.objects.filter(Prefix='ESSD/').all():
        (_, path) = o.key.split('/', 1)
        if not path:
            # Skip ESSD directory all by itself
            continue
        bytes_buffer = io.BytesIO()
        s3_client.download_fileobj(Bucket=bucket.name, Key=o.key, Fileobj=bytes_buffer)
        client.put_file_bytes(commit, "/ESSD/" + path, bytes_buffer.getvalue())

In [ ]:
# Check the file commit
print(list(client.list_file(("essd","master"), "/ESSD/")))